In [ ]:
!wget https://www.dropbox.com/s/n30i1tiwi3s64fg/X-Ray_B.zip?dl=0

In [ ]:
!unzip X-Ray_B.zip?dl=0

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [4]:
train_dir = "X-Ray/train/"
valid_dir = "X-Ray/test/"

img_width, img_height = 224, 224

##ResNet50

In [5]:
conv_base = ResNet50(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))

conv_base.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

## Features Extraction

In [6]:
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 2048))  
    # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='binary')
    
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 1046)  
validation_features, validation_labels = extract_features(valid_dir, 466)

Found 1046 images belonging to 2 classes.
Found 466 images belonging to 2 classes.


In [7]:
train_labels

array([0., 0., 1., ..., 0., 1., 1.])

In [8]:
nsamples1, nx1, ny1 ,nz1= train_features.shape
d2_train_dataset = train_features.reshape((nsamples1,nx1*ny1*nz1))

In [9]:
nsamples, nx, ny ,nz= validation_features.shape
d2_test_dataset = validation_features.reshape((nsamples,nx*ny*nz))

### Implementing SVM

In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc=SVC() 
svc.fit(d2_train_dataset,train_labels)

y_pred=svc.predict(d2_test_dataset)

In [14]:
print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))


Model accuracy score with default hyperparameters: {0:0.4f}: 0.9467


In [11]:
print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))


Model accuracy score with default hyperparameters: 0.6781


### Implementing a Random Forrest

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
newmodel=RandomForestClassifier()

In [15]:
newmodel.fit(d2_train_dataset,train_labels)

RandomForestClassifier()

In [17]:
y_pred=newmodel.predict(d2_test_dataset)
y_pred

array([0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 0., 0.

In [29]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.95      0.94      0.95       236
         1.0       0.94      0.95      0.95       230

    accuracy                           0.95       466
   macro avg       0.95      0.95      0.95       466
weighted avg       0.95      0.95      0.95       466



In [30]:
confusion_matrix(y_pred,validation_labels)

array([[222,  14],
       [ 11, 219]])

### Implementing a KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
knn=KNeighborsClassifier(n_neighbors=7)

In [22]:
knn.fit(d2_train_dataset,train_labels)

KNeighborsClassifier(n_neighbors=7)

In [25]:
y_pred_knn=knn.predict(d2_test_dataset)
y_pred_knn

array([1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
       1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       0., 0., 1., 0., 0.

In [32]:
accuracy_score(y_pred_knn,validation_labels)
print(classification_report(y_pred_knn,validation_labels))

              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90       214
         1.0       0.95      0.88      0.91       252

    accuracy                           0.91       466
   macro avg       0.91      0.91      0.91       466
weighted avg       0.91      0.91      0.91       466



In [34]:
confusion_matrix(y_pred_knn,validation_labels)

array([[202,  12],
       [ 31, 221]])

## Implementing a Decision Tree

In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
dtc=DecisionTreeClassifier()

In [37]:
dtc.fit(d2_train_dataset,train_labels)

DecisionTreeClassifier()

In [38]:
y_pred_dtc=dtc.predict(d2_test_dataset)

In [39]:
y_pred_dtc

array([0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 0., 0.

In [40]:
accuracy_score(y_pred_dtc,validation_labels)
print(classification_report(y_pred_dtc,validation_labels))

              precision    recall  f1-score   support

         0.0       0.91      0.81      0.86       259
         1.0       0.79      0.89      0.84       207

    accuracy                           0.85       466
   macro avg       0.85      0.85      0.85       466
weighted avg       0.86      0.85      0.85       466



In [41]:
confusion_matrix(y_pred_dtc,validation_labels)

array([[211,  48],
       [ 22, 185]])